In [10]:
from __future__ import print_function
import numpy as np
import cntk as C
from cntk.learners import sgd, learning_rate_schedule, UnitType
from cntk.logging import ProgressPrinter
from cntk.layers import Dense, Sequential

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def generate_random_data(sample_size, feature_dim, num_classes):
     # Create synthetic data using NumPy.
     Y = np.random.randint(size=(sample_size, 1), low=0, high=num_classes)

     # Make sure that the data is separable
     X = (np.random.randn(sample_size, feature_dim) + 3) * (Y + 1)
     X = X.astype(np.float32)
     # converting class 0 into the vector "1 0 0",
     # class 1 into vector "0 1 0", ...
     class_ind = [Y == class_number for class_number in range(num_classes)]
     Y = np.asarray(np.hstack(class_ind), dtype=np.float32)
     return X, Y

def ffnet(args):
    lr, act = args
    inputs = 2
    outputs = 2
    layers = 2
    hidden_dimension = 50
    if act == 0:
        activ = C.sigmoid
    elif act == 1:
        activ = C.relu
    elif act == 2:
        activ = C.leaky_relu
        
    # input variables denoting the features and label data
    features = C.input((inputs), np.float32)
    label = C.input((outputs), np.float32)

    # Instantiate the feedforward classification model
    
    my_model = Sequential ([
                    Dense(hidden_dimension, activation=activ),
                    Dense(outputs)])
    z = my_model(features)

    ce = C.cross_entropy_with_softmax(z, label)
    pe = C.classification_error(z, label)

    # Instantiate the trainer object to drive the model training
    lr_per_minibatch = learning_rate_schedule(lr, UnitType.minibatch)
    progress_printer = ProgressPrinter(0)
    trainer = C.Trainer(z, (ce, pe), [sgd(z.parameters, lr=lr_per_minibatch)], [progress_printer])

    # Get minibatches of training data and perform model training
    minibatch_size = 25
    num_minibatches_to_train = 1024
    aggregate_loss = 0.0
    print ("Learning rate ", lr, "; Activation ", activ)
    for i in range(num_minibatches_to_train):
        train_features, labels = generate_random_data(minibatch_size, inputs, outputs)
        # Specify the mapping of input variables in the model to actual minibatch data to be trained with
        trainer.train_minibatch({features : train_features, label : labels})
        sample_count = trainer.previous_minibatch_sample_count
        aggregate_loss += trainer.previous_minibatch_loss_average * sample_count

    last_avg_error = aggregate_loss / trainer.total_number_of_samples_seen

    test_features, test_labels = generate_random_data(minibatch_size, inputs, outputs)
    avg_error = trainer.test_minibatch({features : test_features, label : test_labels})
    print(' aggregate_loss: {}'.format(aggregate_loss))
    return aggregate_loss

np.random.seed(98052)
trials = Trials()
best = fmin(ffnet,
    space=[hp.uniform('lr', 0.01, 0.5), hp.randint('act', 3)],
    algo=tpe.suggest,
    max_evals=20,
    trials=trials)

print (best)

 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate  0.0702697962492488 ; Activation  <function leaky_relu at 0x000000A8C0CCC598>
Learning rate per minibatch: 0.0702697962492488
     2.66       2.66       0.56       0.56            25
     1.89       1.51      0.587        0.6            75
     1.26       0.79      0.554       0.53           175
    0.911      0.603      0.499       0.45           375
    0.755      0.609       0.44      0.385           775
    0.639      0.527      0.342      0.247          1575
    0.526      0.414       0.24       0.14          3175
    0.418       0.31       0.17        0.1          6375
    0.325      0.232      0.126     0.0816         12775
    0.259      0.194     0.0992     0.0728         25575
 aggregate_loss: 6635.911038219929
 average      since    average      since      examples
    loss       last     metric 

    0.937       0.73      0.536      0.485          1575
    0.796      0.657      0.477      0.419          3175
    0.688      0.581      0.398       0.32          6375
    0.527      0.367      0.281      0.165         12775
    0.372      0.218      0.185     0.0881         25575
 aggregate_loss: 9524.383234262466
 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate  0.32335238260381854 ; Activation  <function relu at 0x000000A8C0CCC378>
Learning rate per minibatch: 0.32335238260381854
    0.861      0.861       0.52       0.52            25
     3.67       5.08      0.587       0.62            75
     1.96      0.667       0.52       0.47           175
     1.24      0.621      0.437      0.365           375
    0.916      0.608      0.363      0.292           775
    0.722      0.534      0.318      0.275          1575
    0.628      0.535      0.284   

    0.492      0.385      0.216      0.115          3175
    0.397      0.303      0.163       0.11          6375
     0.31      0.223      0.122     0.0819         12775
    0.254      0.198     0.0986      0.075         25575
 aggregate_loss: 6498.674899876118
 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate  0.23399684458170433 ; Activation  <function leaky_relu at 0x000000A8C0CCC598>
Learning rate per minibatch: 0.23399684458170433
    0.976      0.976       0.24       0.24            25
      1.4       1.62      0.427       0.52            75
     1.18       1.02      0.394       0.37           175
    0.861      0.579       0.36       0.33           375
    0.666      0.484      0.288       0.22           775
    0.533      0.405      0.217      0.149          1575
    0.443      0.355      0.185      0.154          3175
    0.384      0.325      0.